## Email Analysis *Extract, Load, and Transform* Notebook

### Imports

In [ ]:
# Standard Library Imports
import logging
from typing import List, Dict, Any
from datetime import datetime

# Third-Party Imports
import pypff # type: ignore
from sqlalchemy.exc import SQLAlchemyError

# Local Imports
from src.utils.config import Config
from src.extract.pst_message_extractor import PstMessageExtractor
from src.transform.message_parser import MessageParser, ParsedMessage
from src.transform.message_enricher import MessageEnricher, EnrichedMessage
from src.load.data_loader import DataLoader

logging.basicConfig(level=logging.INFO)

### Configuration

In [ ]:
config: Config = Config.from_json("config.json")
extractor: PstMessageExtractor = PstMessageExtractor(config.input_pst_path, config.chunk_size)
message_parser: MessageParser = MessageParser()
message_enricher: MessageEnricher = MessageEnricher()
loader: DataLoader = DataLoader(host=config.db_host, user=config.db_user, password=config.db_password, database=config.db_name)

### ETL Pipeline with Error Handling

In [ ]:
loader.create_tables()

for message_batch in extractor.extract_messages():
    enriched_messages: List[EnrichedMessage] = []
    
    for pst_message in message_batch.messages:
        provider_email_id = pst_message.provider_email_id
        # logging.info(f"Processing message {provider_email_id}")
        try:
            parsed_message: ParsedMessage = message_parser.parse(pst_message.message, provider_email_id, pst_message.folder_name)
        except Exception as e:
            continue

        try:
            enriched_message: EnrichedMessage = message_enricher.enrich_message(parsed_message)
        except Exception as e:
            continue
        else:
            enriched_messages.append(enriched_message)

    try:
        loader.load(enriched_messages)
    except SQLAlchemyError as e:
        logging.error(f"Error loading messages in batch {message_batch.batch_id}: {e}")
        continue
    else:
        logging.info(f"Loaded {len(enriched_messages)} messages in batch {message_batch.batch_id}")

loader.export_to_csv("./data/processed")
loader.close()